<a href="https://colab.research.google.com/github/weagan/Tools/blob/main/hf_to_github_transfer_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Files from Hugging Face to GitHub

This notebook transfers files from `https://huggingface.co/wtfmahe/Samsung-TRM` to `https://github.com/weagan/Samsung-TRM`

**Prerequisites:**
- Add your GitHub Personal Access Token (PAT) to Colab Secrets with the key `GITHUB_PAT`
- Go to: Secrets (🔑 icon in left sidebar) → Add new secret → Name: `GITHUB_PAT`, Value: your token

In [1]:
# Install required packages
!pip install -q huggingface_hub

In [2]:
# Import libraries and get GitHub PAT from secrets
from google.colab import userdata
import os

# Get GitHub PAT from Colab secrets
try:
    GITHUB_PAT = userdata.get('GITHUB_PAT')
    print("✓ GitHub PAT retrieved successfully")
except Exception as e:
    print("❌ Error: Could not retrieve GITHUB_PAT from secrets")
    print("Please add your GitHub Personal Access Token to Colab Secrets with key 'GITHUB_PAT'")
    raise e

✓ GitHub PAT retrieved successfully


In [3]:
# Configure git
!git config --global user.email "colab@example.com"
!git config --global user.name "Colab User"
print("✓ Git configured")

✓ Git configured


In [4]:
# Clone the Hugging Face repository
print("Cloning from Hugging Face...")
!git clone https://huggingface.co/wtfmahe/Samsung-TRM
print("✓ Repository cloned successfully")

Cloning from Hugging Face...
Cloning into 'Samsung-TRM'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 58 (delta 16), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (58/58), 722.67 KiB | 1.10 MiB/s, done.
✓ Repository cloned successfully


In [5]:
# Change to repository directory
%cd Samsung-TRM

# Show current remotes
print("Current remotes:")
!git remote -v

/content/Samsung-TRM
Current remotes:
origin	https://huggingface.co/wtfmahe/Samsung-TRM (fetch)
origin	https://huggingface.co/wtfmahe/Samsung-TRM (push)


In [6]:
# Remove Hugging Face remote and add GitHub remote
print("Removing Hugging Face remote...")
!git remote remove origin

print("Adding GitHub remote...")
# Use PAT in the URL for authentication
github_url = f"https://{GITHUB_PAT}@github.com/weagan/Samsung-TRM.git"
!git remote add origin {github_url}

print("\n✓ Remote updated successfully")
print("New remote (credentials hidden):")
!git remote -v | sed 's/:.*@/:***@/'

Removing Hugging Face remote...
Adding GitHub remote...

✓ Remote updated successfully
New remote (credentials hidden):
origin	https:***@github.com/weagan/Samsung-TRM.git (fetch)
origin	https:***@github.com/weagan/Samsung-TRM.git (push)


In [7]:
# Check current branch
print("Current branch:")
!git branch

# Ensure we're on main branch
!git branch -M main
print("\n✓ Switched to main branch")

Current branch:
* main

✓ Switched to main branch


In [8]:
# Check if remote has content and handle accordingly
print("Checking remote repository status...\n")
!git fetch origin main 2>&1 || echo "Remote branch does not exist yet"

# Try to get remote branch info
import subprocess
result = subprocess.run(['git', 'ls-remote', '--heads', 'origin', 'main'],
                       capture_output=True, text=True)

if result.stdout.strip():
    print("⚠️ Remote repository has existing content.")
    print("Pulling and merging changes...\n")
    !git pull origin main --allow-unrelated-histories --no-edit
    print("\nPushing to GitHub...\n")
    !git push -u origin main
else:
    print("Remote repository is empty. Pushing...\n")
    !git push -u origin main

print("\n✅ Transfer completed successfully!")
print("Your files are now available at: https://github.com/weagan/Samsung-TRM")

Checking remote repository status...

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 3.28 KiB | 1.09 MiB/s, done.
From https://github.com/weagan/Samsung-TRM
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main
⚠️ Remote repository has existing content.
Pulling and merging changes...

From https://github.com/weagan/Samsung-TRM
 * branch            main       -> FETCH_HEAD
hint: You have divergent branches and need to specify how to reconcile them.
hint: You can do so by running one of the following commands sometime before
hint: your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with

## Alternative: Force Push (Use with Caution)

If you want to completely overwrite the GitHub repository content with the Hugging Face content, you can run this cell instead:

**⚠️ WARNING: This will delete any existing content in the GitHub repository!**

In [9]:
# FORCE PUSH - Only use if you want to overwrite GitHub content completely
print("⚠️ Force pushing to GitHub (this will overwrite existing content)...\n")
!git push -u origin main --force

print("\n✅ Force push completed!")
print("Your files are now available at: https://github.com/weagan/Samsung-TRM")

⚠️ Force pushing to GitHub (this will overwrite existing content)...

Enumerating objects: 58, done.
Counting objects: 100% (58/58), done.
Delta compression using up to 2 threads
Compressing objects: 100% (54/54), done.
Writing objects: 100% (58/58), 722.50 KiB | 1.11 MiB/s, done.
Total 58 (delta 17), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (17/17), done.
To https://github.com/weagan/Samsung-TRM.git
 + 19dfdfc...d12141a main -> main (forced update)
Branch 'main' set up to track remote branch 'main' from 'origin'.

✅ Force push completed!
Your files are now available at: https://github.com/weagan/Samsung-TRM


## Troubleshooting

If you encounter errors:

1. **Authentication failed**: Ensure your GitHub PAT has `repo` scope permissions
2. **Repository doesn't exist**: Create the repository at https://github.com/weagan/Samsung-TRM first
3. **Large files**: The notebook automatically handles Git LFS files